# ボートレーサーごとの統計量をdfに追加

## 1. 統計解析用のdfをロード
- `race_df`に格納
- 2_データ前処理.ipnbを参照

In [1]:
import pandas as pd
race_df = pd.read_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl').reset_index(drop=True)
# 中身を確認
race_df.head()

,date,venue,raceNumber,枠,boatNo,motorNo,num_false_start,num_late_start,racer_class,racer_id,...,water_temperature,wave_height,weather,weight,wind_speed,タイム,着順,weather_曇り,weather_雨,weather_雪
0,2021-01-01,津,1.0,1,41,22,0,0,1,4926,...,6.0,2,晴,52.0,3,"1'16""1",2,0,0,0
1,2021-01-01,津,1.0,2,14,57,0,0,3,3644,...,6.0,2,晴,57.1,3,,6,0,0,0
2,2021-01-01,津,1.0,3,46,64,0,0,3,3881,...,6.0,2,晴,52.0,3,,5,0,0,0
3,2021-01-01,津,1.0,4,13,56,0,0,3,4675,...,6.0,2,晴,52.0,3,"1'15""1",1,0,0,0
4,2021-01-01,津,1.0,5,57,63,0,0,3,4358,...,6.0,2,晴,52.0,3,"1'17""2",4,0,0,0


## 2. これまでのレース結果から統計量を算出
- 各選手が、1枠から1着になった確率と2-6枠から3着以内に入った確率を算出し、`rank_ratio`列に格納
- 選手ごと、枠ごと、日付ごとに、その前日までのレースを対象に統計量を算出

In [3]:
from tqdm.notebook import tqdm

def calc_win_ratio(data):
    # 1枠については1着率を算出
    total_count = data.count()
    win_count = (data == 1).sum()
    win_ratio = win_count / total_count
    
    return win_ratio

def calc_in_3_ratio(data):
    # 2枠から6枠については3着以内だった率を算出
    total_count = data.count()
    in_3_count = (data < 4).sum()
    in_3_ratio = in_3_count / total_count
    return in_3_ratio


date_list = race_df["date"].unique()

stocastic_df_list = []

for date in tqdm(date_list):
    race_df_before_the_date = race_df[race_df["date"] < date]

    # data frameを1枠とそれ以外に分ける
    race_df_before_the_date_frame_1 = race_df_before_the_date[race_df_before_the_date["枠"]==1]
    race_df_before_the_date_frame_2_6 = race_df_before_the_date[~(race_df_before_the_date["枠"]==1)]

    # 1枠を集めたdfに対して、選手ごとに一着率を算出
    stocastic_df_1 = race_df_before_the_date_frame_1.groupby("racer_id")["着順"].apply(calc_win_ratio).reset_index()
    stocastic_df_1["枠"] = 1

    # 2-6枠の選手に対しては3着以内に入った率を算出
    stocastoc_df_2_6 = race_df_before_the_date_frame_2_6.groupby(["racer_id", "枠"])["着順"].apply(calc_win_ratio).reset_index()

    # 1枠に対する統計量のdfと2枠以降に対する統計量のdfをconcat
    stocastic_df_date = pd.concat([stocastic_df_1, stocastoc_df_2_6])

    # dfにdate列を追加
    stocastic_df_date["date"] = date

    stocastic_df_list.append(stocastic_df_date)

stocastic_df = pd.concat(stocastic_df_list)
# concatした際にindexgが列になって加わるので消しておく
stocastic_df.drop(columns="index", inplace=True)

# 算出した統計量の列名を変更：rank ratio: 1着についてはその日までの過去レースで1着になった確率、2-6枠については3着以内に入った確率
stocastic_df.rename(columns={'着順': 'rank_ratio'}, inplace=True)

stocastic_df

  0%|          | 0/83 [00:00<?, ?it/s]

Empty DataFrame
Columns: [date, venue, raceNumber, 枠, boatNo, motorNo, num_false_start, num_late_start, racer_class, racer_id, racer_name, ボート2連率, ボート3連率, モーター2連率, モーター3連率, exhibitionTime, exhibition_ST, exhibition_cource, flying, late, temperature, tilt, water_temperature, wave_height, weather, weight, wind_speed, タイム, 着順, weather_曇り, weather_雨, weather_雪]
Index: []

[0 rows x 32 columns]
          date venue  raceNumber  枠  boatNo  motorNo  num_false_start  \
1   2021-01-01   　津　         1.0  2      14       57                0   
2   2021-01-01   　津　         1.0  3      46       64                0   
3   2021-01-01   　津　         1.0  4      13       56                0   
4   2021-01-01   　津　         1.0  5      57       63                0   
5   2021-01-01   　津　         1.0  6      54       11                0   
..         ...   ...         ... ..     ...      ...              ...   
817 2021-01-01   鳴　門        12.0  2      76       65                0   
818 2021-01-01   鳴　門   

KeyboardInterrupt: 

## 3. `race_df`に2で算出した統計量をマージ

In [ ]:
merge_key_list = ["date", "枠", "racer_id"]
race_df_w_stocastic = pd.merge(race_df, stocastic_df,
                               left_on=merge_key_list, right_on=merge_key_list,
                               how="left"
                              )
race_df_w_stocastic

## 4. 3.で作った統計量入りのdfをpickleファイルにして保存

In [ ]:
race_df_w_stocastic.to_pickle('../../data/arrangedData/race_df_w_stocastic.pkl')